In [397]:
def read_graph(file_path):
    adjacency_matrix = []
    adjacency_list = {}
    
    with open(file_path, 'r') as file:
        lines = file.readlines()
        lines.pop(0)
        for i in range(len(lines)):
            lines[i] = lines[i].split()
            lines[i] = [int(x) for x in lines[i]]
        for line in lines:
            adjacency_matrix.append(line)
        for i in range(len(adjacency_matrix)):
            adjacency_list[i] = []
            for j in range(len(adjacency_matrix[i])):
                if adjacency_matrix[i][j] == 1:
                    adjacency_list[i].append(j)
                    

    
    return adjacency_matrix, adjacency_list

adjacency_matrix, adjacency_list = read_graph('grafo.txt')
print("Adjacency Matrix:")
for row in adjacency_matrix:
    print(row)
print("Adjacency List:")
for vertex, neighbors in adjacency_list.items():
    print(f"{vertex}: {neighbors}")


Adjacency Matrix:
[0, 1, 1, 1, 0, 0, 0]
[1, 0, 1, 1, 0, 0, 0]
[1, 1, 0, 1, 1, 1, 0]
[1, 1, 1, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 1, 1]
[0, 0, 1, 0, 1, 0, 1]
[0, 0, 0, 0, 1, 1, 0]
Adjacency List:
0: [1, 2, 3]
1: [0, 2, 3]
2: [0, 1, 3, 4, 5]
3: [0, 1, 2]
4: [2, 5, 6]
5: [2, 4, 6]
6: [4, 5]


In [398]:
def fusion_vertex(adjacency_list, vertex1, vertex2):
    adjacency_list[vertex1].extend(adjacency_list[vertex2])
    adjacency_list.pop(vertex2)
    while vertex2 in adjacency_list[vertex1]:
        adjacency_list[vertex1].remove(vertex2)
    while vertex1 in adjacency_list[vertex1]:
        adjacency_list[vertex1].remove(vertex1)

    for vertex in adjacency_list:
        for neighbor in adjacency_list[vertex]:
            if neighbor == vertex2:
                adjacency_list[vertex][adjacency_list[vertex].index(neighbor)] = vertex1
    
    
    
    
    return adjacency_list


In [399]:
import random
def karger(adjacency_list):
    while len(adjacency_list) > 1:
        # Create a list of edges
        edges = []
        for vertex1, neighbors in adjacency_list.items():
            for vertex2 in neighbors:
                edges.append((vertex1, vertex2))
        
        # Select a random edge
        vertex1, vertex2 = random.choice(edges)
        
        # Check if vertex1 is already a neighbor of vertex2
        if vertex1 in adjacency_list[vertex2]:
            # Call the fusion_vertex function
            adjacency_list = fusion_vertex(adjacency_list, vertex1, vertex2)
    num_edges = len(edges) 
    num_edges = num_edges / 2 
    return num_edges



In [400]:
print(karger(adjacency_list))

3.0


In [401]:
min_return = float('inf') # Initialize with a large value
count_dict = {}
for j in range (100):
    for i in range(6):
        adjacency_matrix, adjacency_list = read_graph('grafo.txt')
        return_value = karger(adjacency_list)
        if return_value < min_return:
            min_return = return_value
    if min_return not in count_dict:
        count_dict[min_return] = 1
    else:
        count_dict[min_return] += 1

print(count_dict)



print(min_return)

{2.0: 100}
2.0
